# Introduction to Chains

- Chains are sequences of operations that transform inputs into outputs.
- They are the fundamental pattern for connecting components in LangChain.
- Chains allow us to focus on composing the flow of logic rather than worrying about the details of execution.
- They syntax of chains makes it easy to visualize the flow of data and logic through a pipeline.

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field

from chain_reaction.config import APIKeys, ModelBehavior, ModelName
from chain_reaction.links import init_model_dump_link

In [ ]:
# Load API keys from .env file
api_keys = APIKeys()

# Initialize a chat model with your API key
chat_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
    **ModelBehavior.factual().model_dump(),
)

# Multi-step workflow

1. Prompt LLM to generate some python code
2. Analyze the code generated from the first prompt with a second LLM call
3. Combine step 1 and 2 to create a complete chain

## 1. Code generation chain

In [ ]:
# Prompt template
python_function_writing_template = ChatPromptTemplate.from_messages([
    ("system", "You are a Python coding assistant adept a writing clean python functions."),
    ("user", "Write a python function that {task_description}"),
])


# Response model
class PythonCode(BaseModel):
    """Python code generation response model."""

    code: str = Field(description="Generated python code.")
    required_libraries: list[str] = Field(description="Required libraries for the code to run.", default_factory=list)
    examples: list[str] = Field(description="Example usages of the generated code.", default_factory=list)
    test_cases: list[str] = Field(description="Test cases to validate the generated code.", default_factory=list)


# Chain
code_gen_chain = python_function_writing_template | chat_model.with_structured_output(PythonCode)

In [ ]:
# Test chain
code_response = code_gen_chain.invoke({
    "task_description": "removes duplicates from a list of integers and returns the sorted result."
})

print("Generated Code:")
print(code_response.code)

print("\nRequired Libraries:")
print(code_response.required_libraries)
print("\nExample Usages:")
for example in code_response.examples:
    print(f"- {example}")

print("\nTest Cases:")
for test_case in code_response.test_cases:
    print(f"- {test_case}")

## 2. Code analysis chain

In [ ]:
# Prompt template
python_analysis_template = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are an expert Python code reviewer versed in modern Python (>= 3.12) best practices.
        You make focused, actionable suggestions to improve code quality, documentation, and tests.""",
    ),
    (
        "user",
        """Please review the following python code and make any recommendations for improvements:\n{code}
        Code requirements: {required_libraries}
        Example usages: {examples}
        Test cases: {test_cases}""",
    ),
])


# Response model
class PythonCodeAnalysis(BaseModel):
    """Code analysis response model."""

    quality_score: int = Field(
        description="An integer score from 1 to 10 representing the overall quality of the code. 10 is the best.",
        ge=1,
        le=10,
    )
    code_improvements: list[str] = Field(
        description="List of suggested improvements to the code.", default_factory=list
    )
    documentation_improvements: list[str] = Field(
        description="List of suggested improvements to the documentation.", default_factory=list
    )
    test_improvements: list[str] = Field(
        description="List of suggested improvements to the test cases.", default_factory=list
    )


# Chain
code_analysis_chain = python_analysis_template | chat_model.with_structured_output(PythonCodeAnalysis)

In [ ]:
# Test chain
analysis_response = code_analysis_chain.invoke(code_response.model_dump())

## 3a. Combine chains

In [ ]:
code_gen_and_analysis_chain = code_gen_chain | init_model_dump_link() | code_analysis_chain

In [ ]:
full_response = code_gen_and_analysis_chain.invoke({
    "task_description": "removes duplicates from a list of integers and returns the sorted result."
})

## 3b. Combine chains & preserve outputs

- If we want to preserve the output from `code_gen_chain`, we can use `RunnablePassthrough` to store the output
- This allows us to maintain context through a chain, rather than passing a single value from start to finish

In [ ]:
code_gen_and_analysis_chain = code_gen_chain | {
    "generated_code": RunnablePassthrough(),  # Preserve the generated code output
    "analysis": init_model_dump_link() | code_analysis_chain,  # also, pipe to analysis chain
}

In [ ]:
full_response = code_gen_and_analysis_chain.invoke({
    "task_description": "removes duplicates from a list of integers and returns the sorted result."
})

full_response.keys()  # dict_keys(['generated_code', 'analysis'])